# Circuit simulation examples

In [1]:
import obi_one as obi
from pathlib import Path

In [2]:
circuit_path_prefix = Path("../../data/tiny_circuits")

### Loading two circuits

In [3]:
circuit_name = "N_10__top_nodes_dim6"
circuit = obi.Circuit(name=circuit_name, path=str(circuit_path_prefix / circuit_name / "circuit_config.json"))
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes[circuit.default_population_name].size} neurons and {circuit.sonata_circuit.edges[circuit.default_edge_population_name].size} synapses")

Circuit 'N_10__top_nodes_dim6' with 10 neurons and 176 synapses


In [4]:
circuit2_name = "N_10__top_rc_nodes_dim2_rc"
circuit2 = obi.Circuit(name=circuit2_name, path=str(circuit_path_prefix / circuit2_name / "circuit_config.json"))
print(f"Circuit '{circuit2}' with {circuit2.sonata_circuit.nodes[circuit2.default_population_name].size} neurons and {circuit2.sonata_circuit.edges[circuit2.default_edge_population_name].size} synapses")

Circuit 'N_10__top_rc_nodes_dim2_rc' with 10 neurons and 19 synapses


### Example with two circuits and explicit node populations.

Spike replay will use POm.

In [5]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(campaign_name="Small microcircuit simulation", campaign_description="Simulation of two small microcircuits with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Timestamps
regular_timesteps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)
sim_conf.add(regular_timesteps, name='stim_times')

# Neuron Sets
node_pop = circuit._default_population_name(circuit.sonata_circuit)

sim_neuron_set = obi.AllNeurons(node_population=node_pop)
stim_neuron_set = sim_neuron_set
replay_neuron_set = sim_neuron_set
rec_neuron_set = sim_neuron_set
property_neuron_set = sim_neuron_set

# sim_neuron_set = obi.PredefinedNeuronSet(node_set="All", node_population=node_pop)
# stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=[50, 100], node_population=node_pop)
# replay_neuron_set = obi.PredefinedNeuronSet(node_set="proj_Thalamocortical_POM_Source", sample_percentage=25, node_population="POm")
# rec_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=100, node_population=node_pop)
# property_neuron_set = obi.PropertyNeuronSet(
#     property_filter=[obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["1", "2", "3"], "synapse_class": ["INH"]}),
#                      obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["4", "5", "6"], "synapse_class": ["EXC"]})],
#     node_population=node_pop
# )
sim_conf.add(sim_neuron_set, name='SimNeurons')
sim_conf.add(stim_neuron_set, name='StimNeurons')
sim_conf.add(rec_neuron_set, name='RecNeurons')
sim_conf.add(property_neuron_set, name='PropertyNeuronSet')
sim_conf.add(replay_neuron_set, name="POM_input")

# Stimuli
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=5.0, neuron_set=stim_neuron_set.ref, amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=100.0, neuron_set=stim_neuron_set.ref, amplitude=0.1)
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timesteps.ref, duration=800, frequency=20,
                                         source_neuron_set=replay_neuron_set.ref,
                                         targeted_neuron_set=stim_neuron_set.ref)
sim_conf.add(current_stimulus, name='CurrentStimulus')
sim_conf.add(sync_current_stimulus, name='SyncCurrentStimulus')
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Voltage Recordings
voltage_recording = obi.SomaVoltageRecording(timestamps=regular_timesteps.ref, neuron_set=rec_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='SomaVoltRec')
                                       
# Initilization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=[circuit, circuit2], node_set=sim_neuron_set.ref, simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [6]:
# simulations_form.model_dump(mode="json")

In [7]:
grid_scan = obi.GridScan(form=validated_sim_conf, output_root='../../../../obi-output/circuit_simulations/grid_scan_explicit')
grid_scan.multiple_value_parameters(display=True)
# grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute()
obi.run_task_for_single_configs(single_configs=grid_scan.single_configs)

[2025-09-24 17:12:33,213] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-09-24 17:12:33,213] INFO: stimuli.CurrentStimulus.amplitude: [0.2, 0.5]
[2025-09-24 17:12:33,213] INFO: initialize.circuit: [Circuit(name='N_10__top_nodes_dim6', path='../../data/tiny_circuits/N_10__top_nodes_dim6/circuit_config.json', matrix_path=None, type='Circuit'), Circuit(name='N_10__top_rc_nodes_dim2_rc', path='../../data/tiny_circuits/N_10__top_rc_nodes_dim2_rc/circuit_config.json', matrix_path=None, type='Circuit')]
[2025-09-24 17:12:33,218] INFO: initialize.circuit is a Circuit instance.
[2025-09-24 17:12:33,248] INFO: initialize.circuit is a Circuit instance.
[2025-09-24 17:12:33,278] INFO: initialize.circuit is a Circuit instance.
[2025-09-24 17:12:33,307] INFO: initialize.circuit is a Circuit instance.


In [8]:
# Spot check of generated spikes files
import h5py

h5 = h5py.File("../../../../obi-output/circuit_simulations/grid_scan_explicit/stimuli.CurrentStimulus.amplitude=0.2/initialize.circuit=N_10__top_nodes_dim6/PoissonInputStimulus_spikes.h5", "r")

print(h5["spikes"].keys())
# print(h5["spikes/POm"]["node_ids"][:])
h5.close()

<KeysViewHDF5 ['S1nonbarrel_neurons']>


### Implicit node populations, i.e. using the default.

Spike replay will use a subset of the default population, in this case: Layer2

In [9]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(campaign_name="Small microcircuit simulation", campaign_description="Simulation of two small microcircuits with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Timestamps
regular_timesteps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)
sim_conf.add(regular_timesteps, name='stim_times')

# Neuron Sets
sim_neuron_set = obi.AllNeurons()
stim_neuron_set = sim_neuron_set
replay_neuron_set = sim_neuron_set
rec_neuron_set = sim_neuron_set
property_neuron_set = sim_neuron_set
# sim_neuron_set = obi.PredefinedNeuronSet(node_set="All")
# stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=[50, 100])
# replay_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=25)
# rec_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=100)
# property_neuron_set = obi.PropertyNeuronSet(
#     property_filter=[obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["1", "2", "3"], "synapse_class": ["INH"]}),
#                      obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["4", "5", "6"], "synapse_class": ["EXC"]})]
# )
sim_conf.add(sim_neuron_set, name='SimNeurons')
sim_conf.add(stim_neuron_set, name='StimNeurons')
sim_conf.add(rec_neuron_set, name='RecNeurons')
sim_conf.add(property_neuron_set, name='PropertyNeuronSet')
sim_conf.add(replay_neuron_set, name="L6_input")

# Stimuli
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=5.0, neuron_set=stim_neuron_set.ref, amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=100.0, neuron_set=stim_neuron_set.ref, amplitude=0.1)
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timesteps.ref, duration=800, frequency=20,
                                         source_neuron_set=replay_neuron_set.ref,
                                         targeted_neuron_set=stim_neuron_set.ref)
sim_conf.add(current_stimulus, name='CurrentStimulus')
sim_conf.add(sync_current_stimulus, name='SyncCurrentStimulus')
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Voltage Recordings
voltage_recording = obi.SomaVoltageRecording(timestamps=regular_timesteps.ref, neuron_set=rec_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='SomaVoltRec')
                                       
# Initilization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=[circuit, circuit2], node_set=sim_neuron_set.ref, simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [10]:
grid_scan = obi.GridScan(form=validated_sim_conf, output_root='../../../../obi-output/circuit_simulations/grid_scan_implicit')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute()
obi.run_task_for_single_configs(single_configs=grid_scan.single_configs)

[2025-09-24 17:12:33,345] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-09-24 17:12:33,345] INFO: stimuli.CurrentStimulus.amplitude: [0.2, 0.5]
[2025-09-24 17:12:33,345] INFO: initialize.circuit: [Circuit(name='N_10__top_nodes_dim6', path='../../data/tiny_circuits/N_10__top_nodes_dim6/circuit_config.json', matrix_path=None, type='Circuit'), Circuit(name='N_10__top_rc_nodes_dim2_rc', path='../../data/tiny_circuits/N_10__top_rc_nodes_dim2_rc/circuit_config.json', matrix_path=None, type='Circuit')]
[2025-09-24 17:12:33,345] INFO: 
COORDINATE PARAMETERS
[2025-09-24 17:12:33,345] INFO: stimuli.CurrentStimulus.amplitude: 0.2, initialize.circuit: N_10__top_nodes_dim6
[2025-09-24 17:12:33,346] INFO: stimuli.CurrentStimulus.amplitude: 0.2, initialize.circuit: N_10__top_rc_nodes_dim2_rc
[2025-09-24 17:12:33,346] INFO: stimuli.CurrentStimulus.amplitude: 0.5, initialize.circuit: N_10__top_nodes_dim6
[2025-09-24 17:12:33,346] INFO: stimuli.CurrentStimulus.amplitude: 0.5, initialize.circuit: N_10__top_rc_n

In [11]:
# Spot check of generated spikes files
import h5py

h5 = h5py.File("../../../../obi-output/circuit_simulations/grid_scan_explicit/stimuli.CurrentStimulus.amplitude=0.2/initialize.circuit=N_10__top_nodes_dim6/PoissonInputStimulus_spikes.h5", "r")

print(h5["spikes"].keys())
print(h5["spikes/S1nonbarrel_neurons"]["node_ids"][:])
h5.close()

<KeysViewHDF5 ['S1nonbarrel_neurons']>
[ 2 10  7  5  3  3  1  8  5  9  4  6  1  8  1  1  3  8  6 10  2  1  3  2
  9  8  7  3 10  2  1  4  6  5  3  2  9  4  1  9  6  6  5  9  1 10 10  3
  7  9  8  6  8 10 10  2  4  4  6  2 10  8  8  4  2  1  6  6  4  3  9  3
  5  7  9  4  5  2  2  8  2  3  4 10  8  5  4  4  9  2  7  6  3 10  7  8
  8  2  9  9  8  5  8  4  9  3  9  6  4  2  8  4  9  5  3  7  2  4  3  5
  2  7  8  1  4  9  8  5  7  4  5  8 10  3  4 10  3  8  4  8  8  3  2  1
  8  4  7  5  7  5  2  5  3 10  4  5  9  5  3  8  6  5  4  1  3  9  2  3
  4  5  1 10  7  3  5  1  6  5  4  7  1  4  2  7  5  1  9  5  4  8  2  4
  4  3  2  9  9  5 10  2  6 10  8 10  4  5  3  3  9  4 10  1  3  4  6  6
  2 10  1  5  8  3  5  9  9  1  1  1 10  9  3  8  6  9  2  2  5  6  9  2
  3  9  7  4  7  8  6 10 10  3  7  7  1  9  7  3  4  4  4  7  3  6  4  8
  3  9  7  7  7  9  1  2  5  4  3  8  1  9 10  4 10  3  5  2  9  3  9  2
  3  2  6  1  8  5  4  6  6  4  3  9  7 10  1  2  8 10  4  4 10  4  2  6
  1  9  8  3

### Implicit node populations, but still using an extrinsic population for replay.

This is achieved using a hard coded VPM neuron set.

In [12]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(campaign_name="Small microcircuit simulation", campaign_description="Simulation of two small microcircuits with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Timestamps
regular_timesteps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=1000.0)
sim_conf.add(regular_timesteps, name='stim_times')

# Neuron Sets
sim_neuron_set = obi.AllNeurons()
stim_neuron_set = sim_neuron_set
rec_neuron_set = sim_neuron_set
property_neuron_set = sim_neuron_set
# sim_neuron_set = obi.PredefinedNeuronSet(node_set="All")
# stim_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=[50, 100])
replay_neuron_set = obi.nbS1VPMInputs(sample_percentage=25)
# rec_neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=100)
# property_neuron_set = obi.PropertyNeuronSet(
#     property_filter=[obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["1", "2", "3"], "synapse_class": ["INH"]}),
#                      obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["4", "5", "6"], "synapse_class": ["EXC"]})]
# )
sim_conf.add(sim_neuron_set, name='SimNeurons')
sim_conf.add(stim_neuron_set, name='StimNeurons')
sim_conf.add(rec_neuron_set, name='RecNeurons')
sim_conf.add(property_neuron_set, name='PropertyNeuronSet')
sim_conf.add(replay_neuron_set, name="VPM_input")

# Stimuli
current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=5.0, neuron_set=stim_neuron_set.ref, amplitude=[0.2, 0.5])
sync_current_stimulus = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timesteps.ref, duration=100.0, neuron_set=stim_neuron_set.ref, amplitude=0.1)
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timesteps.ref, duration=800, frequency=20,
                                         source_neuron_set=replay_neuron_set.ref,
                                         targeted_neuron_set=stim_neuron_set.ref)
sim_conf.add(current_stimulus, name='CurrentStimulus')
sim_conf.add(sync_current_stimulus, name='SyncCurrentStimulus')
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Voltage Recordings
voltage_recording = obi.SomaVoltageRecording(timestamps=regular_timesteps.ref, neuron_set=rec_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='SomaVoltRec')
                                       
# Initilization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=[circuit, circuit2], node_set=sim_neuron_set.ref, simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [13]:
grid_scan = obi.GridScan(form=validated_sim_conf, output_root='../../../../obi-output/circuit_simulations/grid_scan_vpm_hardcoded')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)
grid_scan.execute()
obi.run_task_for_single_configs(single_configs=grid_scan.single_configs)

[2025-09-24 17:12:33,480] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-09-24 17:12:33,480] INFO: stimuli.CurrentStimulus.amplitude: [0.2, 0.5]
[2025-09-24 17:12:33,480] INFO: initialize.circuit: [Circuit(name='N_10__top_nodes_dim6', path='../../data/tiny_circuits/N_10__top_nodes_dim6/circuit_config.json', matrix_path=None, type='Circuit'), Circuit(name='N_10__top_rc_nodes_dim2_rc', path='../../data/tiny_circuits/N_10__top_rc_nodes_dim2_rc/circuit_config.json', matrix_path=None, type='Circuit')]
[2025-09-24 17:12:33,480] INFO: 
COORDINATE PARAMETERS
[2025-09-24 17:12:33,481] INFO: stimuli.CurrentStimulus.amplitude: 0.2, initialize.circuit: N_10__top_nodes_dim6
[2025-09-24 17:12:33,481] INFO: stimuli.CurrentStimulus.amplitude: 0.2, initialize.circuit: N_10__top_rc_nodes_dim2_rc
[2025-09-24 17:12:33,481] INFO: stimuli.CurrentStimulus.amplitude: 0.5, initialize.circuit: N_10__top_nodes_dim6
[2025-09-24 17:12:33,481] INFO: stimuli.CurrentStimulus.amplitude: 0.5, initialize.circuit: N_10__top_rc_n

In [14]:
# Spot check of generated spikes files
import h5py

h5 = h5py.File("../../../../obi-output/circuit_simulations/grid_scan_vpm_hardcoded/stimuli.CurrentStimulus.amplitude=0.2/initialize.circuit=N_10__top_nodes_dim6/PoissonInputStimulus_spikes.h5", "r")

print(h5["spikes"].keys())
print(h5["spikes/VPM"]["node_ids"][:])
h5.close()

<KeysViewHDF5 ['VPM']>
[  8 191  29 ... 160 106  29]


In [15]:
# Deserialization
# grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../../../obi-output/circuit_simulations/grid_scan_vpm_hardcoded/run_scan_config.json")

In [16]:
# grid_scan_ds